In [6]:
import decoupler as dc
from pathlib import Path
import os
import nichecompass as nc
import pandas as pd
import numpy as np

# Explore different databases
From Niche compass we can query:
- omnipath lr interactions
- collectri tf networks
- mebocost es interactions
- nichenet lrt interactions

From Decoupler we can query omnipath with the following resources:

In [2]:
dc.show_resources()

['Adhesome',
 'Almen2009',
 'Baccin2019',
 'CORUM_Funcat',
 'CORUM_GO',
 'CPAD',
 'CSPA',
 'CSPA_celltype',
 'CancerDrugsDB',
 'CancerGeneCensus',
 'CancerSEA',
 'CellCall',
 'CellCellInteractions',
 'CellChatDB',
 'CellChatDB_complex',
 'CellPhoneDB',
 'CellPhoneDB_complex',
 'CellTalkDB',
 'CellTypist',
 'Cellinker',
 'Cellinker_complex',
 'ComPPI',
 'CytoSig',
 'DGIdb',
 'DisGeNet',
 'EMBRACE',
 'Exocarta',
 'GO_Intercell',
 'GPCRdb',
 'Guide2Pharma',
 'HGNC',
 'HPA_secretome',
 'HPA_subcellular',
 'HPA_tissue',
 'HPMR',
 'HumanCellMap',
 'ICELLNET',
 'ICELLNET_complex',
 'IntOGen',
 'Integrins',
 'InterPro',
 'KEGG',
 'KEGG-PC',
 'Kirouac2010',
 'LOCATE',
 'LRdb',
 'Lambert2018',
 'MCAM',
 'MSigDB',
 'Matrisome',
 'MatrixDB',
 'Membranome',
 'NetPath',
 'OPM',
 'PROGENy',
 'PanglaoDB',
 'Phobius',
 'Phosphatome',
 'Ramilowski2015',
 'Ramilowski_location',
 'SIGNOR',
 'SignaLink_function',
 'SignaLink_pathway',
 'Surfaceome',
 'TCDB',
 'TFcensus',
 'TopDB',
 'UniProt_family',
 'UniP

## Let's start with gene programs from Niche compass

### Omnipath apy for lr interactions of mouse organism

In [8]:
lr_interactions = nc.utils.extract_gp_dict_from_omnipath_lr_interactions(
    species="mouse",
    gene_orthologs_mapping_file_path=Path(os.getcwd()).parents[1] / "data" / "raw" / "human_mouse_gene_orthologs.csv",
    plot_gp_gene_count_distributions=False,
)
print(type(lr_interactions))

<class 'dict'>


In [38]:
lr_df = pd.DataFrame.from_dict(lr_interactions, orient='index').reset_index()
lr_df.columns = ['lr_name', 'sources', 'target', 'sources_categories', 'targets_categories']
lr_df = lr_df.set_index('lr_name')
lr_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1042 entries, EPOR_ligand_receptor_GP to FCN1_ligand_receptor_GP
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sources             1042 non-null   object
 1   target              1042 non-null   object
 2   sources_categories  1042 non-null   object
 3   targets_categories  1042 non-null   object
dtypes: object(4)
memory usage: 40.7+ KB


In [39]:
lr_df

,sources,target,sources_categories,targets_categories
lr_name,,,,
EPOR_ligand_receptor_GP,[EPOR],[STAT3],[ligand],[receptor]
FYN_ligand_receptor_GP,[FYN],[PIK3CD],[ligand],[receptor]
KL_ligand_receptor_GP,[KL],[FGFR1],[ligand],[receptor]
S100A10_ligand_receptor_GP,[S100A10],[PLA2G4A],[ligand],[receptor]
JAG2_ligand_receptor_GP,[JAG2],[NOTCH4],[ligand],[receptor]
...,...,...,...,...
CHRD_ligand_receptor_GP,[CHRD],[BMPR2],[ligand],[receptor]
COMPLEX:IL6ST_LIFR_ligand_receptor_GP,[COMPLEX:IL6ST_LIFR],"[JAK1, JAK2, TYK2]",[ligand],"[receptor, receptor, receptor]"
ANGPTL5_ligand_receptor_GP,[ANGPTL5],[TEK],[ligand],[receptor]


## Extract liana consensus LR database

In [51]:
import liana as li
li.resource.show_resources()

['baccin2019',
 'cellcall',
 'cellchatdb',
 'cellinker',
 'cellphonedb',
 'celltalkdb',
 'connectomedb2020',
 'consensus',
 'embrace',
 'guide2pharma',
 'hpmr',
 'icellnet',
 'italk',
 'kirouac2010',
 'lrdb',
 'mouseconsensus',
 'ramilowski2015']

In [52]:
lr_consensus = li.resource.select_resource("mouseconsensus")

In [48]:
lr_consensus

,ligand,receptor
0,LGALS9,PTPRC
1,LGALS9,MET
2,LGALS9,CD44
3,LGALS9,LRP1
4,LGALS9,CD47
...,...,...
4619,BMP2,ACTR2
4620,BMP15,ACTR2
4621,CSF1,CSF3R
4622,IL36G,IFNAR1


## Extract nichenet lrt interactions

In [88]:
lrt_interactions = nc.utils.extract_gp_dict_from_nichenet_lrt_interactions(
    species="mouse",
    gene_orthologs_mapping_file_path=Path(os.getcwd()).parents[0] / "data" / "raw" / "human_mouse_gene_orthologs.csv",
    plot_gp_gene_count_distributions=False,
)

/home/bio/PhD/projects/mlnetst/.venv/lib/python3.12/site-packages/nichecompass/utils/gene_programs.py:496: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.


In [89]:
lrt_df = pd.DataFrame.from_dict(lrt_interactions, orient='index').reset_index()
lrt_df.columns = ['lrt_name', 'sources', 'targets', 'sources_categories', 'targets_categories']
lrt_df = lrt_df.set_index('lrt_name')
lrt_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1287 entries, 2300002M23Rik_ligand_receptor_target_gene_GP to Zpbp2_ligand_receptor_target_gene_GP
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sources             1287 non-null   object
 1   targets             1287 non-null   object
 2   sources_categories  1287 non-null   object
 3   targets_categories  1287 non-null   object
dtypes: object(4)
memory usage: 50.3+ KB


### Reading nichenet weighted networks for mouse from resource etracted from main nichenet repo

In [92]:
gr_df = pd.read_csv(Path(os.getcwd()).parents[0] / "data" / "raw" / "gr.csv",)
lr_sig_df = pd.read_csv(Path(os.getcwd()).parents[0] / "data" / "raw" / "lr_sig.csv",)

print(gr_df.info())
print(lr_sig_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364411 entries, 0 to 4364410
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   from    object 
 1   to      object 
 2   weight  float64
dtypes: float64(1), object(2)
memory usage: 99.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3865137 entries, 0 to 3865136
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   from    object 
 1   to      object 
 2   weight  float64
dtypes: float64(1), object(2)
memory usage: 88.5+ MB
None


## Extract mebocost es interactions

In [24]:
es_interactions = nc.utils.extract_gp_dict_from_mebocost_es_interactions(
    species="mouse",
    plot_gp_gene_count_distributions=False,
    dir_path=str(Path(os.getcwd()).parents[0] / "data" / "raw")
)

In [27]:
es_df = pd.DataFrame.from_dict(es_interactions, orient='index').reset_index()
es_df.columns = ['es_name', 'sources', 'sources_categories', 'targets', 'targets_categories']
es_df = es_df.set_index('es_name')
es_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137 entries, Deoxyuridine_metabolite_enzyme_sensor_GP to Iron_metabolite_enzyme_sensor_GP
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sources             137 non-null    object
 1   sources_categories  137 non-null    object
 2   targets             137 non-null    object
 3   targets_categories  137 non-null    object
dtypes: object(4)
memory usage: 5.4+ KB


## Extract niche compass gene program idea behind collectri TF network

In [28]:
tf_network = nc.utils.extract_gp_dict_from_collectri_tf_network(
    species="mouse",
    plot_gp_gene_count_distributions=False,
)

/home/bio/PhD/projects/mlnetst/.venv/lib/python3.12/site-packages/liana/resource/_orthology.py:199: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


In [29]:
tf_df = pd.DataFrame.from_dict(tf_network, orient='index').reset_index()
tf_df.columns = ['tf_name', 'sources', 'sources_categories', 'targets', 'targets_categories']
tf_df = tf_df.set_index('tf_name')
tf_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1096 entries, AP1_TF_target_genes_GP to Zxdc_TF_target_genes_GP
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sources             1096 non-null   object
 1   sources_categories  1096 non-null   object
 2   targets             1096 non-null   object
 3   targets_categories  1096 non-null   object
dtypes: object(4)
memory usage: 42.8+ KB


# Extract decoupler collectri

In [30]:
net = dc.get_collectri(
    organism = "mouse",
    split_complexes=False,
)

/home/bio/PhD/projects/mlnetst/.venv/lib/python3.12/site-packages/liana/resource/_orthology.py:199: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


In [33]:
net.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36867 entries, 0 to 36866
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  36867 non-null  string
 1   target  36867 non-null  string
 2   weight  36867 non-null  Int64 
dtypes: Int64(1), string(2)
memory usage: 900.2 KB


# Example of a data that we can intersect with the features of these metaresources

In [34]:
import anndata as ad
x_hat_s = ad.read_h5ad(Path(os.getcwd()).parents[0] / "data" / "processed" / "mouse1_slice153_x_hat_s.h5ad")

In [63]:
lr_df = lr_consensus.copy()
lr_df.info()

lr_df_exploded = li.resource.explode_complexes(lr_df, SOURCE="ligand", TARGET="receptor")
lr_df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3989 entries, 31371 to 35359
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ligand    3989 non-null   object
 1   receptor  3989 non-null   object
dtypes: object(2)
memory usage: 93.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5075 entries, 0 to 5074
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   interaction       5075 non-null   object
 1   ligand            5075 non-null   object
 2   receptor          5075 non-null   object
 3   ligand_complex    5075 non-null   object
 4   receptor_complex  5075 non-null   object
dtypes: object(5)
memory usage: 198.4+ KB


In [64]:
lr_df["ligand"] = lr_df["ligand"].str.lower()
lr_df["receptor"] = lr_df["receptor"].str.lower()
print(
    len(set(x_hat_s.var_names).intersection(set(lr_df["ligand"]))), "/", len(x_hat_s.var_names), " | ", len(set(lr_df["ligand"]))
    )
print(
    len(set(x_hat_s.var_names).intersection(set(lr_df["receptor"]))), "/", len(x_hat_s.var_names), " | ", len(set(lr_df["receptor"]))
    )


792 / 18450  |  878
733 / 18450  |  907


In [65]:
lr_df_exploded["ligand"] = lr_df_exploded["ligand"].str.lower()
lr_df_exploded["receptor"] = lr_df_exploded["receptor"].str.lower()
print(
    len(set(x_hat_s.var_names).intersection(set(lr_df_exploded["ligand"]))), "/", len(x_hat_s.var_names), " | ", len(set(lr_df_exploded["ligand"]))
    )
print(
    len(set(x_hat_s.var_names).intersection(set(lr_df_exploded["receptor"]))), "/", len(x_hat_s.var_names), " | ", len(set(lr_df_exploded["receptor"]))
    )


800 / 18450  |  873
771 / 18450  |  795


In [96]:
import squidpy as sq
class_labels = x_hat_s.obs["class_label"].astype("category").cat.categories
subclass_labels = x_hat_s.obs["subclass"].astype("category").cat.categories
print(subclass_labels)

x_hat_s.obsm["spatial"] = np.array([(x,y) for x,y in zip(x_hat_s.obs["centroid_x"], x_hat_s.obs["centroid_y"])])
for label in subclass_labels:
    subsample_x_hat_s = x_hat_s[x_hat_s.obs["subclass"] == label]
    if subsample_x_hat_s.shape[0] <100:
        continue

Index(['Astro', 'Endo', 'L2/3 IT', 'L4/5 IT', 'L5 ET', 'L5 IT', 'L5/6 NP',
       'L6 CT', 'L6 IT', 'L6 IT Car3', 'L6b', 'Lamp5', 'Micro', 'OPC', 'Oligo',
       'PVM', 'Peri', 'Pvalb', 'SMC', 'Sncg', 'Sst', 'VLMC', 'Vip', 'other'],
      dtype='object')


In [83]:
import omnipath as op
resource = op.interactions.PostTranslational.get()

11.7MB [00:00, 23.7MB/s]


In [97]:
gr_df.info()
gr_df["from"] = gr_df["from"].str.lower()
gr_df["to"] = gr_df["to"].str.lower()

SUBCLASSES_TO_EXPLORE = ["Astro", "L2/3 IT"]
x_hat_s_subsampled = x_hat_s[x_hat_s.obs["subclass"].isin(SUBCLASSES_TO_EXPLORE)]
print(
    len(set(x_hat_s_subsampled.var_names).intersection(set(gr_df["from"]))), "/", len(x_hat_s_subsampled.var_names), " | ", len(set(gr_df["from"]))
    )
print(
    len(set(x_hat_s_subsampled.var_names).intersection(set(gr_df["to"]))), "/", len(x_hat_s_subsampled.var_names), " | ", len(set(gr_df["to"]))
    )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364411 entries, 0 to 4364410
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   from    object 
 1   to      object 
 2   weight  float64
dtypes: float64(1), object(2)
memory usage: 99.9+ MB
4583 / 18450  |  5411
16649 / 18450  |  22164


In [137]:
import scipy
def get_expressed_genes(adata, pct):
    n_cells_in_matrix = adata.shape[0]
    # Calculate proportions for all genes at once using numpy operations
    proportion = (adata.X > 0.1).sum(axis=0) / n_cells_in_matrix
    # For sparse matrix, need to convert to array
    if scipy.sparse.issparse(proportion):
        proportion = proportion.A1
    # Get indices where proportion >= pct
    genes = adata.var_names[proportion >= pct]
    return genes


In [110]:
all_receptors = set(gr_df["to"])
expressed_genes_receiver = get_expressed_genes(subsample_x_hat_s, 0.1)
expressed_receptors = all_receptors.intersection(expressed_genes_receiver)
potential_ligands = gr_df.loc[gr_df["to"].isin(expressed_receptors),"from"].unique()

In [138]:
potential_ligands = {}
for i, cell_type in enumerate(SUBCLASSES_TO_EXPLORE):
    print("Iteration {} for celltype {}".format(i, cell_type))
    subsample_x_hat_s = x_hat_s[x_hat_s.obs["subclass"] == cell_type]
    print("\tDimension of subsample {}".format(subsample_x_hat_s.shape))
    expressed_genes_receiver = get_expressed_genes(subsample_x_hat_s, 0.1)
    expressed_receptors = all_receptors.intersection(expressed_genes_receiver)
    potential_ligands[cell_type] = gr_df.loc[gr_df["to"].isin(expressed_receptors),"from"].unique()

Iteration 0 for celltype Astro
	Dimension of subsample (537, 18450)
Iteration 1 for celltype L2/3 IT
	Dimension of subsample (1178, 18450)


In [142]:
print(
    len(set(potential_ligands["Astro"]).intersection(set(potential_ligands["L2/3 IT"]))),
    "/", len(potential_ligands["L2/3 IT"]), " | ", len(set(potential_ligands["Astro"]))
)

4611 / 4827  |  4622
